In [ ]:
import json
import urllib.parse
import urllib.request
import base64
from datetime import datetime
import pandas as pd

# Stałe
service_url = 'xxxx'
user_login = 'xxx'            # <----------------------------------- zmiana loginu
user_password = 'xxx'       # <----------------------------------- zmiana hasła na swoje

auth_string = f'{user_login}:{user_password}'.encode('utf-8')
auth_base64 = base64.b64encode(auth_string).decode('utf-8')
headers = {
    'Authorization': f'Basic {auth_base64}',
    'Content-Type': 'application/json'
}

# Wczytanie listy stacji
stations_df = pd.read_csv('stacje_do_SPEI_92.csv', encoding='cp1250')   # <--------------plik ze stacjami (jesli znajduję się w jakiejś innej lokalizacji trzeba wpisać całą ścieżke)
stations_df["id"] = stations_df["id"].astype(str)

# Funkcja pobierająca dane z SH API
def fetch_data(station_ids, parameter):
    values = {
        'dre-names': ','.join(station_ids),
        'parameter-ids': parameter,
        'from': '20250501000000',   # <------------------------------- zmiana daty 
        'to':   '20250531235900',   # <------------------------------- zmiana daty   (musi byc w tej postaci)
        'time-specification': '10'
    }
    url_values = urllib.parse.urlencode(values)
    full_url = f'{service_url}?{url_values}'
    req = urllib.request.Request(full_url, headers=headers, method='GET')
    try:
        with urllib.request.urlopen(req) as response:
            content = response.read()
            return json.loads(content)
    except:
        return []

# Funkcja przetwarzająca dane do DataFrame
def process_records(records):
    data = []
    for record in records:
        try:
            station_id = record["dre-name"]
            date = datetime.strptime(str(record["date"]), '%Y%m%d%H%M%S')
            temp_value = float(record["value"])
            data.append([station_id, date, temp_value])
        except:
            continue
    if not data:
        return pd.DataFrame()
    df = pd.DataFrame(data, columns=["station_id", "timestamp", "temperature"])
    df["date"] = df["timestamp"].dt.date
    df["hour"] = df["timestamp"].dt.strftime('%H:%M')

    grouped = df.groupby(["station_id", "date"])["temperature"].agg(["min", "max", "mean"]).reset_index()
    temps_at_hours = df[df["hour"].isin(["06:00", "18:00"])]
    pivoted = temps_at_hours.pivot_table(index=["station_id", "date"], 
                                         columns="hour", 
                                         values="temperature", 
                                         aggfunc="first").reset_index()
    pivoted.columns.name = None
    final_df = pd.merge(grouped, pivoted, on=["station_id", "date"], how="left")
    final_df["mean"] = final_df["mean"].round(2)
    
    return final_df

# === Krok 1: Pobierz dane z B00304A ===
station_ids = stations_df["id"].tolist()
records_main = fetch_data(station_ids, "B00302A")
df_main = process_records(records_main)

# Zidentyfikuj użyte stacje
if not df_main.empty and "station_id" in df_main.columns:
    used_ids = df_main["station_id"].astype(str).unique().tolist()
    remaining_ids = [sid for sid in station_ids if sid not in used_ids]
else:
    print("⚠️ df_main jest pusty lub brak kolumny 'station_id'. Pobieram dane dla wszystkich stacji.")
    remaining_ids = station_ids.copy()
    df_main = pd.DataFrame()  # upewnia się, że df_main istnieje dalej jako pusty

# === Krok 2: Pobierz dane z B00302A dla reszty ===
records_fallback = fetch_data(remaining_ids, "B00304A")
df_fallback = process_records(records_fallback)

# === Krok 3: Połącz dane i dołącz stacje ===
final_df = pd.concat([df_main, df_fallback], ignore_index=True)
final_df["station_id"] = final_df["station_id"].astype(str)
stations_df["id"] = stations_df["id"].astype(str)

merged_df = pd.merge(final_df, stations_df[["id", "Nazwa.stacji", "rank_code"]],
                     left_on="station_id", right_on="id", how="left")
merged_df.drop(columns=["station_id"], inplace=True)

merged_df.rename(columns={
"06:00": "godz_6",
"18:00": "godz_18"
}, inplace=True)

# === Zapis do CSV ===  <----------------- zmien nazwe pliku lub ściezke
merged_df.to_csv("xxx", index=False, encoding='utf-8-sig')
print("✅ Dane zapisane")
merged_df

In [ ]:
merged_df.head(60)

In [ ]:
len(merged_df.id.unique().tolist())   # informacja ile jest unikalnych stacji